The features we engineer were actually inspired from [this](https://www.kaggle.com/mhviraf/a-new-baseline-for-dsb-2019-catboost-model) kaggle kernel. The idea is to look at the various activities of a person before an assessment: such as the number and types of different clips they watched, the number and types of different games they played, the number and types of different assesments they took, their accuracy on these assessments and so on. 

Recall that specs.csv contains information that describes the various events in any game_session. We will not be using this to engineer our features. Thus, in order to save memory, we will not load specs.csv for the time being.

 **List of features to create**


1) number of clips watched before attempting the assessment: Note that there are 20 different titles under the type 'clips'. We will create one entry for each such title. Additionally, one more entry for just the total number of clips watched (before the assessment) irrespective of their title. => 21 features.

2) number of activities done before the assessment: 8 different activities + 1 for all the activities = 9 features. 

3) number of games played before the assessment: 11 different games + 1 for all the games = 12 features

4) number of assessments done before the current assessment: 5 different assessments + 1 for all the assessments = 6 features

5) total (accumalated) amount of time spent watching different clips before the assessment: 20 different clips + 1 for all the clips = 21 features

6) total time spent on activities: 8 different activities + 1 for all the activities = 9 features

7) total time spent on games: 11 different games + 1 for all the games = 12

8) total time spent on previous assessments: 5 different assessments + 1 for all the assessments

9) average event_count in the clips watched (before the assessment): 20+1= 21 features

10) average event_count in activities (before the assessment): 8+1 = 9 features

11) average event_count in games (before the assessment): 11+1=12 features

12) average event_count in previous assessments: 5+1=6 features

13) average accuracy in previous assessments: 5+1 = 6 features


In total, we therefore have 48 x 3 + 6 = 150 features in the above list

Also note that we should only consider game_session which lasted for a non-zero time interval. Indeed there are game_session that were abandoned immediately after starting, for e.g. the session with id = '45bb1e1b6b50c07b'. These should not be included in the statistics being assembled above.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

In [2]:
print('The current working directory is: {}'.format(os.getcwd()))

The current working directory is: C:\Users\agarw\Dropbox\Kaggle data-science-bowl 2019\Prarit-data-science-bowl-2019


In [3]:
train=pd.read_csv('train.csv')

In [4]:
train.shape

(11341042, 11)

In [5]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [6]:
trainlbls=pd.read_csv('train_labels.csv')

In [7]:
trainlbls.shape

(17690, 7)

In [8]:
trainlbls.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [9]:
# merging the data in train and trainlbls according to game_session, installation_id and title
train=train.merge(trainlbls, how='left', on=['game_session', 'installation_id', 'title'])
train.shape

(11341042, 15)

In [10]:
train.keys()

Index(['event_id', 'game_session', 'timestamp', 'event_data',
       'installation_id', 'event_count', 'event_code', 'game_time', 'title',
       'type', 'world', 'num_correct', 'num_incorrect', 'accuracy',
       'accuracy_group'],
      dtype='object')

In [11]:
# for purposes of testing, we will first use only a small amount of data first. So we 
# choose to work with the data for only 10 installation ids. 

#id_chosen=train.installation_id.unique()[0:10]

In [12]:
# the installation_ids chosen above
#id_chosen

In [13]:
# reducing the train dataframe size by restricting to only 10 installation ids as chosen above
# use the pd.Series.isin() to choose rows based on a given list
# this was suggested in the following stackexchange post:
# https://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
#train=train.loc[train.installation_id.isin(id_chosen)]

In [14]:
# shape of the reduced dataframe
#train.shape

In [15]:
# converting the timestamp string to datetime format
# for documentation on pandas datetime functionality, see
# see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html 
train['datetime']=pd.to_datetime(train.timestamp)
train.drop(columns=['timestamp'], inplace=True)

In [16]:

train.head(10)

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group,datetime
0,27253bdc,45bb1e1b6b50c07b,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,2019-09-06 17:53:46.937
1,27253bdc,17eeb7f223665f53,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:17.519
2,77261ab5,0848ef14a8dc6892,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.302
3,b2dba42b,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:54:56.387
4,1bb5fbdb,0848ef14a8dc6892,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:03.253
5,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.279
6,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:06.913
7,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.546
8,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:07.979
9,1325467d,0848ef14a8dc6892,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-09-06 17:55:08.566


In [17]:
# function to compute the total time (in ms) spent and number of events in a game_session

def game_session_time_events(game_session=None, dataframe = train):
    if not game_session:
        print('no game_session was passed to the function')
        return 
    sess=dataframe.loc[dataframe.game_session==game_session]
    
    # time spent (in ms)
    session_time= sess.datetime 
    start_time=session_time.min()
    end_time=session_time.max()
    time_interval=pd.Timedelta(end_time-start_time).delta/10**6
    
    # number of events
    num_events=sess.event_count.max()
    
    
    return time_interval, num_events

In [18]:
game_session_time_events()

no game_session was passed to the function


In [19]:
game_session_time_events('0848ef14a8dc6892')

(194307.0, 267)

In [20]:
# function to count previous game_sessions of a given type and title

def count_prev_sessions(dataframe, current_time, typ, title):
    '''dataframe: the dataframe containing the record of a particular player
       current_time: the time at which the assessment (whose statistics we are looking for) started'''
    
    # locating game_sessions of the specified type and title that occurred before current_time
    earlier_sessions=dataframe.loc[(dataframe.datetime<current_time) 
                                   & (dataframe.type==typ) & (dataframe.title==title)].game_session.unique()
    
    # selecting sessions with non-zero session_time
    session_time_events=[game_session_time_events(session) for session in earlier_sessions]
    good_sessions=[ (session, time_events[0], time_events[1]) for session, time_events in zip(earlier_sessions, session_time_events) if time_events[0]>0]
    
    num_good_sessions=len(good_sessions)
    
    # The following methodology to obtain the n-th tuple element in a list of tuples is
    # based on suggestion given in the following stackexchange post
    # https://stackoverflow.com/questions/12142133/how-to-get-first-element-in-a-list-of-tuples
    cummulative_time=0
    avg_n_events=0
    if num_good_sessions>0: 
        cummulative_time=np.sum(list(list(zip(*good_sessions))[1]))
        avg_n_events=np.mean(list(list(zip(*good_sessions))[2]))
    
    
    avg_accuracy=0
    mean_accuracy_group=0
    
    if typ=='Assessment' and num_good_sessions>0:
        # good_sess_result_dat is the dataset containing statistics for assessments that are considered good
        # basically we are choosing the rows in trainlbls whose game_session is in goodsessions
        # use the pd.Series.isin() to choose rows based on a given list
        # this was suggested in the following stackexchange post:
        # https://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
        good_sess_results_dat= trainlbls.loc[
            trainlbls.game_session.isin(list(list(zip(*good_sessions))[0]))]
        avg_accuracy=good_sess_results_dat.accuracy.mean()
        mean_accuracy_group=good_sess_results_dat.accuracy_group.mean()
    
    
    return earlier_sessions, good_sessions, num_good_sessions, cummulative_time, avg_n_events, avg_accuracy, mean_accuracy_group

In [21]:
stTm=train.loc[train.game_session=='901acc108f55a5a1'].datetime.min()
player=train.loc[train.game_session=='901acc108f55a5a1'].installation_id.iloc[0]
print('The assessment start time for player with installation_id {} is {}'.format(player,stTm))

The assessment start time for player with installation_id 0006a69f is 2019-08-06 05:22:01.344000


In [22]:
all_prev_sess, good_sess,num_good_sess, cumm_time, avg_n_ev, avg_acc, avg_acc_grp=count_prev_sessions(dataframe=train.loc[train.installation_id=='0006a69f'],
                                                                                                      current_time=stTm, typ='Activity', title='Sandcastle Builder (Activity)')

In [23]:
train.loc[(train.installation_id=='0006a69f') & (train.datetime<stTm)]

,event_id,game_session,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group,datetime
1538,27253bdc,34ba1a28d02ba8ba,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,NaN,NaN,NaN,NaN,2019-08-06 04:57:18.904
1539,27253bdc,4b57c9a59474a1b9,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:57:45.301
1540,77261ab5,2b9d5af79bcdb79f,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:14.538
1541,b2dba42b,2b9d5af79bcdb79f,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:14.615
1542,1325467d,2b9d5af79bcdb79f,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:16.680
1543,1325467d,2b9d5af79bcdb79f,"{""coordinates"":{""x"":863,""y"":237,""stage_width"":...",0006a69f,4,4070,3937,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:18.474
1544,1325467d,2b9d5af79bcdb79f,"{""coordinates"":{""x"":817,""y"":617,""stage_width"":...",0006a69f,5,4070,4820,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:19.365
1545,1bb5fbdb,2b9d5af79bcdb79f,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,6,3110,6954,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:21.490
1546,1325467d,2b9d5af79bcdb79f,"{""coordinates"":{""x"":809,""y"":180,""stage_width"":...",0006a69f,7,4070,8187,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:22.732
1547,5e812b27,2b9d5af79bcdb79f,"{""size"":0,""coordinates"":{""x"":782,""y"":207,""stag...",0006a69f,8,4030,8745,Sandcastle Builder (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,2019-08-06 04:58:23.295


Some important comments: 

1) Need to think what will I do for 'assessment' sessions where an attempt was never made. One way is to completely remove them from the data -> for training set this does not pose a problem because trainlbls only contains the results of completed assessments. Since we directly use trainlbls to obtain the assessment results, therefore incomplete assessments don't event appear. -> However, they will still contribute to other statistics such as number of assessments and avg. number of events per assessment as well as cummulative time spent on assessments. Perhaps this is how it should be done. 

2) results of assessments in the test data are not given explicity. Will have to check if these can also be obtained from trainlbls or will need to be extracted from event_data?

3) In the function, count_prev_sessions when I compute avg. accuracy or accuracy_group_mode, I need to only choose one row per good_session. The way I am doing it right now, it chooses all the rows in good_session.This implies the accuracy and accuracy_group of a session are considered n_events times. Here n_events is the number of events in the session. This might not be a problem for avg. accuracy but the accuracy_group_mode can and will be incorrectly swayed by sessions with high event_count. -> Again because I am using trainlbls directly to obtain the assessment results, this will not be a problem since trainlbls only contains one row per session. 

In [24]:
# list of various different clips, activities etc in the app

clip_list=[('Clip', title) for title in train.loc[train.type=='Clip'].title.unique()]

activity_list=[('Activity', title) for title in train.loc[train.type=='Activity'].title.unique()]

game_list=[('Game', title) for title in train.loc[train.type=='Game'].title.unique()]

assessment_list=[('Assessment', title) 
                 for title in train.loc[train.type=='Assessment'].title.unique()]

type_title_list=clip_list+activity_list+game_list+assessment_list

print('There are a total of {} different clips/games/activities/assessment in the app'.
      format(len(type_title_list)))

There are a total of 44 different clips/games/activities/assessment in the app


In [25]:
num_features=len(type_title_list)*3 + len(assessment_list) + 4*3 + 1
print('We expect to build {} features through the above function'.format(num_features))

We expect to build 150 features through the above function


Now apply count_prev_sessions to each session in trainlbls. 

In [27]:
# The following function enables us to apply count_prev_sess to each row of trainlbs

features=[]

def apply_count_prev_sess(row):
    session=row.game_session
    player=row.installation_id
    acc_grp=row.accuracy_group
    dataframe=train.loc[train.installation_id==player]
    current_time=dataframe.loc[dataframe.game_session==session].datetime.min()
    session_title=row.title
    
    # getting statistics for all the earlier sessions
    stats=[player, session, session_title,  acc_grp]
    for typ, title in type_title_list:
        _, _,num_good_sess, cumm_time, avg_n_ev, avg_acc, avg_acc_grp=count_prev_sessions(dataframe, current_time, typ, title)
        lis=[typ, title, num_good_sess, cumm_time, avg_n_ev, avg_acc, avg_acc_grp]
        stats.append(lis)
    
    features.append(stats)

In [33]:
%%time
# for testing purposes only applying 'apply_count_prev_sess' to the first 2 rows of trainlbls
# finally will need to run the loop for the whole length of trainlbls
# length of trainlbls can be obtained from trainlbls.shape[0]

# features is the array that will contain the features for each assessment session
features=[]
trainlbls.iloc[0:2].apply(lambda x: apply_count_prev_sess(x), axis=1)

Wall time: 45.9 s


0    None
1    None
dtype: object

In [ ]:
# for testing purposes only doing this for first 2 rows of trainlbls
# finally will need to run the loop for the whole length of trainlbls
# length of trainlbls can be obtained from trainlbls.shape[0]

# features is the array that will contain the features for each assessment session
features=[]

for idx in range(trainlbls.shape[0]):
    row=trainlbls.iloc[idx]
    session=row.game_session
    player=row.installation_id
    acc_grp=row.accuracy_group
    dataframe=train.loc[train.installation_id==player]
    current_time=dataframe.loc[dataframe.game_session==session].datetime.min()
    session_title=row.title
    
    # getting statistics for all the earlier sessions
    stats=[player, session, session_title,  acc_grp]
    for typ, title in type_title_list:
        _, _,num_good_sess, cumm_time, avg_n_ev, avg_acc, avg_acc_grp=count_prev_sessions(dataframe, current_time, typ, title)
        lis=[typ, title, num_good_sess, cumm_time, avg_n_ev, avg_acc, avg_acc_grp]
        stats.append(lis)
    
    features.append(stats)

In [35]:
trainlbls.shape[0]

17690

In [43]:
features[0]

['0006a69f',
 '6bdf9623adc94d89',
 'Mushroom Sorter (Assessment)',
 3,
 ['Clip', 'Welcome to Lost Lagoon!', 0, 0, 0, 0, 0],
 ['Clip', 'Magma Peak - Level 1', 0, 0, 0, 0, 0],
 ['Clip', 'Magma Peak - Level 2', 0, 0, 0, 0, 0],
 ['Clip', 'Tree Top City - Level 1', 0, 0, 0, 0, 0],
 ['Clip', 'Ordering Spheres', 0, 0, 0, 0, 0],
 ['Clip', 'Slop Problem', 0, 0, 0, 0, 0],
 ['Clip', 'Costume Box', 0, 0, 0, 0, 0],
 ['Clip', '12 Monkeys', 0, 0, 0, 0, 0],
 ['Clip', 'Tree Top City - Level 2', 0, 0, 0, 0, 0],
 ['Clip', "Pirate's Tale", 0, 0, 0, 0, 0],
 ['Clip', 'Treasure Map', 0, 0, 0, 0, 0],
 ['Clip', 'Tree Top City - Level 3', 0, 0, 0, 0, 0],
 ['Clip', 'Rulers', 0, 0, 0, 0, 0],
 ['Clip', 'Crystal Caves - Level 1', 0, 0, 0, 0, 0],
 ['Clip', 'Balancing Act', 0, 0, 0, 0, 0],
 ['Clip', 'Crystal Caves - Level 2', 0, 0, 0, 0, 0],
 ['Clip', 'Crystal Caves - Level 3', 0, 0, 0, 0, 0],
 ['Clip', 'Lifting Heavy Things', 0, 0, 0, 0, 0],
 ['Clip', 'Honey Cake', 0, 0, 0, 0, 0],
 ['Clip', 'Heavy, Heavier, Heaviest

In [37]:
# arr is an array containing the a specified set of stats for all assessment sessions
# for eg if we want the stats corresponding 'clip' titled 'Magma Peak - Level 1' for all assessments
# then we notice that the corresdong list is the 6th entry in 'features', so we give the command: np.array(list(zip(*features))[5]
feat=pd.DataFrame([])

for itr in range(4,48):
    arr=np.array(list(zip(*features))[itr])
    typ=arr[0,0]
    title=arr[0,1]
    if not typ=='Assessment':
        dat=pd.DataFrame(arr[::, 2:5], columns=['{}_{}_num_good_sess'.format(typ, title), 
                                       '{}_{}_cumm_time'.format(typ, title), 
                                       '{}_{}_avg_n_ev'.format(typ, title)])
    else:
        dat=pd.DataFrame(arr[::, 2:], columns=['{}_{}_num_good_sess'.format(typ, title), 
                                       '{}_{}_cumm_time'.format(typ, title), 
                                       '{}_{}_avg_n_ev'.format(typ, title),
                                       '{}_{}_avg_acc'.format(typ, title), 
                                       '{}_{}_avg_acc_grp'.format(typ, title)])
        
    feat=pd.concat([feat, dat], axis = 1)

In [38]:
feat

,Clip_Welcome to Lost Lagoon!_num_good_sess,Clip_Welcome to Lost Lagoon!_cumm_time,Clip_Welcome to Lost Lagoon!_avg_n_ev,Clip_Magma Peak - Level 1_num_good_sess,Clip_Magma Peak - Level 1_cumm_time,Clip_Magma Peak - Level 1_avg_n_ev,Clip_Magma Peak - Level 2_num_good_sess,Clip_Magma Peak - Level 2_cumm_time,Clip_Magma Peak - Level 2_avg_n_ev,Clip_Tree Top City - Level 1_num_good_sess,...,Assessment_Cart Balancer (Assessment)_num_good_sess,Assessment_Cart Balancer (Assessment)_cumm_time,Assessment_Cart Balancer (Assessment)_avg_n_ev,Assessment_Cart Balancer (Assessment)_avg_acc,Assessment_Cart Balancer (Assessment)_avg_acc_grp,Assessment_Chest Sorter (Assessment)_num_good_sess,Assessment_Chest Sorter (Assessment)_cumm_time,Assessment_Chest Sorter (Assessment)_avg_n_ev,Assessment_Chest Sorter (Assessment)_avg_acc,Assessment_Chest Sorter (Assessment)_avg_acc_grp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
feat['assessment_title']=np.array(list(zip(*features))[2])

In [41]:
feat['accuracy_grp']=list(list(zip(*features))[3])

In [42]:
feat

,Clip_Welcome to Lost Lagoon!_num_good_sess,Clip_Welcome to Lost Lagoon!_cumm_time,Clip_Welcome to Lost Lagoon!_avg_n_ev,Clip_Magma Peak - Level 1_num_good_sess,Clip_Magma Peak - Level 1_cumm_time,Clip_Magma Peak - Level 1_avg_n_ev,Clip_Magma Peak - Level 2_num_good_sess,Clip_Magma Peak - Level 2_cumm_time,Clip_Magma Peak - Level 2_avg_n_ev,Clip_Tree Top City - Level 1_num_good_sess,...,Assessment_Cart Balancer (Assessment)_avg_n_ev,Assessment_Cart Balancer (Assessment)_avg_acc,Assessment_Cart Balancer (Assessment)_avg_acc_grp,Assessment_Chest Sorter (Assessment)_num_good_sess,Assessment_Chest Sorter (Assessment)_cumm_time,Assessment_Chest Sorter (Assessment)_avg_n_ev,Assessment_Chest Sorter (Assessment)_avg_acc,Assessment_Chest Sorter (Assessment)_avg_acc_grp,assessment_title,accuracy_grp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Mushroom Sorter (Assessment),3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bird Measurer (Assessment),0


In [51]:
len(feat.keys()[0:59:3])

20